# 如何使用内置工具和工具包

:::info 先决条件

本指南假设您已熟悉以下概念：

- [LangChain 工具](/docs/concepts/tools)
- [LangChain 工具包](/docs/concepts/tools)

:::

## 工具

LangChain 拥有大量第三方工具。请访问 [工具集成](/docs/integrations/tools/) 查看可用工具列表。

:::important

使用第三方工具时，请确保您了解该工具的工作原理及其拥有的权限。请仔细阅读其文档，并从安全角度检查是否需要您进行任何操作。请参阅我们的 [安全](https://python.langchain.com/docs/security/) 指南了解更多信息。

:::

让我们来试用一下 [Wikipedia 集成](/docs/integrations/tools/wikipedia/)。

In [1]:
!pip install -qU langchain-community wikipedia

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 


该工具有以下与之关联的默认设置：

In [3]:
print(f"Name: {tool.name}")
print(f"Description: {tool.description}")
print(f"args schema: {tool.args}")
print(f"returns directly?: {tool.return_direct}")

Name: wikipedia
Description: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
args schema: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
returns directly?: False


## 自定义默认工具

我们还可以修改内置的名称、描述和参数的 JSON schema。

在定义参数的 JSON schema 时，重要的是输入保持与函数相同，因此你不应该更改它。但是，你可以轻松地为每个输入定义自定义描述。

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )


tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

print(tool.run("langchain"))

Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 


In [5]:
print(f"Name: {tool.name}")
print(f"Description: {tool.description}")
print(f"args schema: {tool.args}")
print(f"returns directly?: {tool.return_direct}")

Name: wiki-tool
Description: look up things in wikipedia
args schema: {'query': {'description': 'query to look up in Wikipedia, should be 3 or less words', 'title': 'Query', 'type': 'string'}}
returns directly?: True


## 如何使用内置工具包

工具包是为特定任务协同设计的工具集合。它们提供了便捷的加载方法。

所有工具包都暴露一个 `get_tools` 方法，该方法返回一个工具列表。

通常的使用方式如下：

```python
# 初始化一个工具包
toolkit = ExampleTookit(...)

# 获取工具列表
tools = toolkit.get_tools()
```